In [5]:
from preprocessing import get_features_and_target
import tensorflow as tf

# LSTM regression on returns

# Preprocessing regression

In [8]:
symbol = "BTC/USDT"
feature_lags = []
valid_split = 200
train_length = 5
batch_size = 32
seed = 44

df = get_features_and_target(
    symbol,
    days_to_forecast=1,
    feature_lags=feature_lags,
    model_type="reg",
    model_freq="1d",
)

symbol = symbol.replace("/", ":")

X = df.drop(columns=f"{symbol}_target")

y = df[f"{symbol}_target"].copy()

X_train, X_valid, y_train, y_valid = (
    X[:-valid_split],
    X[-valid_split:],
    y[:-valid_split],
    y[-valid_split:],
)

train_df = tf.keras.utils.timeseries_dataset_from_array(
    X_train,
    targets=y_train[(train_length - 1) :],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=False,
    seed=seed,
)

valid_df = tf.keras.utils.timeseries_dataset_from_array(
    X_valid,
    targets=y_valid[(train_length - 1) :],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=False,
    seed=seed,
)

# Check
# Initialize variables to hold the first batch's data
# Get the first batch from the dataset
for X_batch, y_batch in train_df.take(1):
    # Extract the first sample from the batch
    first_sample_features = X_batch[0][-1].numpy().tolist()
    first_sample_target = y_batch[0]

    # Print the feature vector and its corresponding target
    print("Feature Vector (First Sample):", first_sample_features)
    print("Target (First Sample):", first_sample_target)
    break  # Exit after processing the first batch

X_train.iloc[47], y_train[47]
y_train

y.describe()

Feature Vector (First Sample): [10735.26222222222, 10547.578000000003, 10116.574200000003, 8880.152299999998, 41.17816467648574, 11857.22687345585, 9237.929126544157, 16.82102462307948, -77.51678820056262, -138991.55009900004, 10385.0, 14.78948862926205, 39.56579879394895, 100.0, 78.57142857142857]
Target (First Sample): tf.Tensor(-0.004814682192147202, shape=(), dtype=float64)


/var/folders/cd/xc6lw60d5711s9k_4hjp_dwc0000gn/T/ipykernel_6219/781309817.py:60: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  X_train.iloc[47], y_train[47]


count    1982.000000
mean        0.001304
std         0.036504
min        -0.395048
25%        -0.014467
50%         0.000343
75%         0.016730
max         0.195361
Name: BTC:USDT_target, dtype: float64

# Model Regression

In [8]:
import tensorflow as tf
from tensorflow.keras import regularizers

class LSTMregression(tf.keras.Model):
    def __init__(self, num_units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)

        self.norm_layer = tf.keras.layers.Normalization()
        self.conv1 = tf.keras.layers.Conv1D(
            filters=num_units * 2,
            kernel_size=2,
            strides=1,
            padding="causal",
            activation=activation,
        )
        self.lstm1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(num_units, return_sequences=True, stateful=False)
        )
        self.lstm2 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(num_units, return_sequences=True, stateful=False)
        )
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.dense = tf.keras.layers.Dense(
            num_units,
            activation=activation,
            kernel_initializer="he_normal",
            kernel_regularizer=regularizers.l2(0.01),
        )
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.output_layer = tf.keras.layers.Dense(1)

    def call(self, inputs):
        x = self.norm_layer(inputs)
        x = self.conv1(x)
        x = self.lstm1(x)
        x = self.layer_norm(x)
        x = self.lstm2(x)
        x = self.layer_norm(x)
        x = self.dense(x)
        x = self.dropout(x)
        return self.output_layer(x)


model = LSTMregression()

# Adapt normalization layer
model.norm_layer.adapt(X_train)

# Compile the model
model.compile(optimizer="adam", loss="mse", metrics=["mae", "RootMeanSquaredError"])

In [ ]:
# Callbacks
tf.keras.backend.clear_session()
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, monitor="val_mae", restore_best_weights=True
)
tensorboard = tf.keras.callbacks.TensorBoard("TB_regression_logs")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_mae",
    factor=0.5,  # Reduce learning rate by half
    patience=1,  # Number of epochs with no improvement
    min_lr=0.0001,  # Minimum learning rate
)
history = model.fit(
    train_df,
    validation_data=valid_df,
    callbacks=[early_stopping, tensorboard, reduce_lr],
    epochs=50,
)

In [11]:
model.save("last_regression_model.keras")  # Save regression the model

# Classification model

# Preprocessing classificaton

In [25]:
symbol = "BTC/USDT"
feature_lags = []
valid_split = 2000
train_length = 48
batch_size = 32
seed = 44

df_classification = get_features_and_target(
    symbol,
    days_to_forecast=1,
    feature_lags=feature_lags,
    model_type="class",
)

symbol = symbol.replace("/", ":")

X = df_classification.drop(columns=f"{symbol}_target")

y = df_classification[f"{symbol}_target"].copy().map(lambda x: int(x))

X_train, X_valid, y_train, y_valid = (
    X[:-valid_split],
    X[-valid_split:],
    y[:-valid_split],
    y[-valid_split:],
)

train_df_classification = tf.keras.utils.timeseries_dataset_from_array(
    X_train,
    targets=y_train[train_length:],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=True,  # Shaffles the sequences, not within sequences
    seed=seed,
)

valid_df_classification = tf.keras.utils.timeseries_dataset_from_array(
    X_valid,
    targets=y_valid[train_length:],
    sequence_length=train_length,
    batch_size=batch_size,
    shuffle=True,
    seed=seed,
)

for X_batch, y_batch in train_df_classification.take(1):
    print("Train batch X shape:", X_batch)
    print("Train batch y shape:", y_batch)  # starts from y.iloc[48]

print(f"X_shape = {X_batch.shape}")
print(f"y_shape = {y.shape}")

Train batch X shape: tf.Tensor(
[[[3.98791444e+03 3.99111850e+03 3.99332180e+03 ... 4.12691418e+01
   9.28571429e+01 3.57142857e+01]
  [3.98707444e+03 3.99132450e+03 3.99361620e+03 ... 4.48170716e+01
   8.57142857e+01 2.85714286e+01]
  [3.98757889e+03 3.99173250e+03 3.99387920e+03 ... 5.60380796e+01
   7.85714286e+01 2.14285714e+01]
  ...
  [3.93456889e+03 3.94889050e+03 3.97370180e+03 ... 4.02517141e+01
   2.85714286e+01 0.00000000e+00]
  [3.93641000e+03 3.94567800e+03 3.97233600e+03 ... 2.73721019e+01
   2.14285714e+01 0.00000000e+00]
  [3.93529667e+03 3.94265750e+03 3.97118800e+03 ... 2.21275047e+01
   1.42857143e+01 0.00000000e+00]]

 [[9.36813667e+03 9.34704100e+03 9.33306280e+03 ... 2.03941418e+01
   0.00000000e+00 6.42857143e+01]
  [9.36539000e+03 9.35106200e+03 9.33216940e+03 ... 2.80374919e+01
   8.57142857e+01 5.71428571e+01]
  [9.36295444e+03 9.35489250e+03 9.33143260e+03 ... 3.60592276e+01
   1.00000000e+02 5.00000000e+01]
  ...
  [9.64090000e+03 9.61000900e+03 9.46923360e+

In [45]:
import tensorflow as tf
from tensorflow.keras import regularizers


class LSTMclassification(tf.keras.Model):
    def __init__(self, num_classes, num_units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)

        self.norm_layer = tf.keras.layers.Normalization()
        self.conv1 = tf.keras.layers.Conv1D(
            filters=num_units * 2,
            kernel_size=2,
            strides=1,
            padding="causal",
            activation=activation,
        )
        self.lstm1 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(num_units, return_sequences=True, stateful=False)
        )
        self.lstm2 = tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(num_units, return_sequences=False, stateful=False)
        )
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.dense = tf.keras.layers.Dense(
            num_units,
            activation=activation,
            kernel_initializer="he_normal",
            kernel_regularizer=regularizers.l2(0.01),
        )
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.output_layer = tf.keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        x = self.norm_layer(inputs)
        x = self.conv1(x)
        x = self.lstm1(x)
        x = self.layer_norm(x)
        x = self.lstm2(x)
        x = self.dense(x)
        x = self.dropout(x)
        return self.output_layer(x)


# Assuming df, X_train, and num_classes are already defined
num_features = len(df.columns) - 1
model = LSTMclassification(num_classes=3, num_units=30)

# Adapt normalization layer
model.norm_layer.adapt(X_train)

model.build(input_shape=(batch_size, train_length, num_features))

# Compile the model
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall()],
)

model.summary()

Model: "lst_mclassification_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_13 (Normaliz  multiple                  31        
 ation)                                                          
                                                                 
 conv1d_13 (Conv1D)          multiple                  1860      
                                                                 
 bidirectional_26 (Bidirect  multiple                  21840     
 ional)                                                          
                                                                 
 bidirectional_27 (Bidirect  multiple                  21840     
 ional)                                                          
                                                                 
 layer_normalization_13 (La  multiple                  120       
 yerNormalization)                          

In [46]:
from sklearn.utils import class_weight
tf.keras.backend.clear_session()

class_weights = class_weight.compute_class_weight(
    "balanced", classes=np.unique(y_train), y=y_train
)

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=3, monitor="accuracy", restore_best_weights=True
)

tensorboard = tf.keras.callbacks.TensorBoard("TB_classification_logs")
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="accuracy",
    factor=0.5,  # Reduce learning rate by half
    patience=1,  # Number of epochs with no improvement
    min_lr=0.0001,  # Minimum learning rate
)
model.fit(
    train_df_classification,
    validation_data=valid_df_classification,
    callbacks=[early_stopping, tensorboard, reduce_lr],
    epochs=1,
    class_weight=class_weights,
)

ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py", line 1249, in compute_metrics

In [ ]:
model.save("last_classification_model.keras")